In [3]:
import pandas as pd
import numpy as np
import joblib
from sklearn.neighbors import NearestNeighbors
import os
import pandas as pd

In [4]:
import pandas as pd

In [5]:
dataset = pd.read_csv('tourist_places_sri_lanka.csv')

In [3]:
print(os.getcwd())

<class 'NameError'>: name 'os' is not defined

In [4]:
import os

In [8]:
print(os.getcwd())

/drive


In [5]:
dataset.head()

,Place Name,Latitude,Longitude,Cost,Categories,Sub Places
0,Place_1,7.591243,80.925954,14209,"Nature, Beach, Wildlife","Nine Arches Bridge, Little Adam's Peak, Miriss..."
1,Place_2,6.890975,79.710062,12907,"Adventure, Nature","Yala National Park, Polonnaruwa Ruins"
2,Place_3,7.564333,81.675479,16741,"Religious, Wildlife",Udawalawe National Park
3,Place_4,8.676660,81.508631,13833,"Cultural, Nature",Nine Arches Bridge
4,Place_5,9.498860,80.065566,12820,Cultural,"Mirissa Beach, Udawalawe National Park, Gregor..."


In [10]:
dataset.shape

(1000, 6)

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Place Name  1000 non-null   object 
 1   Latitude    1000 non-null   float64
 2   Longitude   1000 non-null   float64
 3   Cost        1000 non-null   int64  
 4   Categories  1000 non-null   object 
 5   Sub Places  1000 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 35.2+ KB


In [12]:
# Viwe statical mesurement about the dataset
dataset.describe()

,Latitude,Longitude,Cost
count,1000.000000,1000.000000,1000.000000
mean,7.908170,80.782749,12595.309000
std,1.118261,0.627788,6128.076353
min,5.902463,79.701349,1490.000000
25%,6.964826,80.244666,7613.000000
50%,7.947222,80.782348,13725.000000
75%,8.857511,81.315535,16802.750000
max,9.794452,81.887096,86414.000000


In [13]:
dataset["Cost"].value_counts()

Cost
4000     4
6000     4
7000     4
16838    3
12729    2
        ..
14203    1
12494    1
8661     1
9163     1
12138    1
Name: count, Length: 954, dtype: int64

In [14]:
dataset["Categories"].value_counts()

Categories
Adventure                      61
Nature                         61
Religious                      57
Cultural                       55
Beach                          55
                               ..
Beach, Religious, Cultural      1
Religious, Nature, Wildlife     1
Cultural, Nature, Religious     1
Beach, Adventure, Religious     1
Wildlife, Cultural, Beach       1
Name: count, Length: 152, dtype: int64

In [15]:
dataset["Longitude"].value_counts()

Longitude
80.925954    1
80.413885    1
81.082268    1
79.859445    1
80.212805    1
            ..
81.746158    1
80.424681    1
81.612194    1
80.997834    1
81.283336    1
Name: count, Length: 1000, dtype: int64

In [16]:
dataset["Latitude"].value_counts()

Latitude
7.591243    1
8.611673    1
6.609560    1
7.149952    1
6.549230    1
           ..
6.080355    1
8.775533    1
9.412752    1
8.562904    1
8.559834    1
Name: count, Length: 1000, dtype: int64

In [17]:
dataset["Place Name"].value_counts()

Place Name
Place_1       1
Place_672     1
Place_659     1
Place_660     1
Place_661     1
             ..
Place_339     1
Place_340     1
Place_341     1
Place_342     1
Place_1000    1
Name: count, Length: 1000, dtype: int64

In [1]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [32]:
import joblib 

In [33]:
def train_and_save_model(dataset):
    # Calculate distance from mean latitude and longitude
    dataset['distance'] = dataset.apply(
        lambda row: np.sqrt(
            (row['Latitude'] - dataset['Latitude'].mean()) ** 2 + 
            (row['Longitude'] - dataset['Longitude'].mean()) ** 2
        ),
        axis=1
    )

    # Train the KNN model
    knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
    knn.fit(dataset[['distance', 'normalized_cost']])
    
    # Save the trained model
    joblib.dump(knn, 'knn_model.pkl')
    print("Model saved as knn_model.pkl")

# Train and save the model
train_and_save_model(dataset)

Model saved as knn_model.pkl


In [12]:
import numpy as np

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [24]:
from sklearn.neighbors import NearestNeighbors

In [34]:
knn_model = joblib.load('knn_model.pkl')
print("Model loaded from knn_model.pkl")

Model loaded from knn_model.pkl


In [35]:
print(knn_model)

NearestNeighbors(metric='euclidean')


In [36]:
# Example user location and budget
user_location = [6.931, 79.990]  # Example lat, long
budget = 5000  # Example budget

# Calculate distance for the user from all places
dataset['distance'] = dataset.apply(
    lambda row: np.sqrt(
        (row['Latitude'] - user_location[0]) ** 2 + 
        (row['Longitude'] - user_location[1]) ** 2
    ),
    axis=1
)

# Calculate relevance based on cost and distance
dataset['relevance'] = (
    dataset['normalized_cost'] * (budget / 10000) +
    dataset['distance'] * 1  # Adjust this weight as needed
)

# Filter places based on the budget and other criteria
filtered_data = dataset[dataset['Cost'] <= budget]

# Make recommendations
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(filtered_data[['distance', 'normalized_cost']])
_, indices = knn.kneighbors(filtered_data[['distance', 'normalized_cost']])

# Get recommendations
recommendations = filtered_data.iloc[indices[0]].to_dict(orient='records')
print(recommendations)



[{'Place Name': 'Place_11', 'Latitude': 9.630933, 'Longitude': 80.537002, 'Cost': 2500, 'Categories': 'Nature, Religious, Beach', 'Sub Places': 'Galle Fort', 'distance': 2.7547866328434605, 'normalized_cost': 0.011892986670434742, 'relevance': 2.760733126178678}, {'Place Name': 'Place_867', 'Latitude': 9.379991, 'Longitude': 81.271309, 'Cost': 2259, 'Categories': 'Beach, Cultural, Adventure', 'Sub Places': 'Horton Plains', 'distance': 2.7639301133643053, 'normalized_cost': 0.009055155197588432, 'relevance': 2.7684576909630993}, {'Place Name': 'Place_508', 'Latitude': 9.185475, 'Longitude': 81.543291, 'Cost': 2257, 'Categories': 'Beach, Cultural, Religious', 'Sub Places': 'Mirissa Beach, Temple of the Tooth', 'distance': 2.737767421879734, 'normalized_cost': 0.009031604728934106, 'relevance': 2.742283224244201}, {'Place Name': 'Place_586', 'Latitude': 8.904433, 'Longitude': 81.88602, 'Cost': 2124, 'Categories': 'Wildlife, Adventure', 'Sub Places': 'Sigiriya Museum', 'distance': 2.736663

In [37]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [23]:
def train_and_save_model(df):
    # Calculate distance from mean latitude and longitude
    df['distance'] = df.apply(
        lambda row: np.sqrt(
            (row['Latitude'] - df['Latitude'].mean()) ** 2 + 
            (row['Longitude'] - df['Longitude'].mean()) ** 2
        ),
        axis=1
    )

    # Train the KNN model
    knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
    knn.fit(df[['distance', 'normalized_cost']])

    # Make predictions on the training data
    distances, indices = knn.kneighbors(df[['distance', 'normalized_cost']])

    # For each sample, calculate the mean cost of the nearest neighbors
    predicted_costs = []
    for i in range(len(df)):
        # Get the 'Cost' values for the nearest neighbors for each sample
        neighbor_costs = df.iloc[indices[i]]['Cost']
        # Calculate the mean cost of the nearest neighbors
        predicted_costs.append(neighbor_costs.mean())

    # Convert predicted_costs list to a numpy array for easier calculations
    predicted_costs = np.array(predicted_costs)

    # Compute the Mean Squared Error (MSE) and Mean Absolute Error (MAE) for the training data
    mse = mean_squared_error(df['Cost'], predicted_costs)
    mae = mean_absolute_error(df['Cost'], predicted_costs)

    print(f"Mean Squared Error (MSE) on training data: {mse}")
    print(f"Mean Absolute Error (MAE) on training data: {mae}")
    
    # Calculate accuracy (percent error)
    accuracy = 100 - (mse / df['Cost'].mean()) * 100
    print(f"Training accuracy: {accuracy:.2f}%")

    # Save the trained model to a specified file path
    # file_path = 'knn_model.pkl'
    # joblib.dump(knn, file_path)
    print(f"Model saved at ")


In [24]:
train_and_save_model(dataset)

Mean Squared Error (MSE) on training data: 3358174.5100799995
Mean Absolute Error (MAE) on training data: 527.716
Training accuracy: -26562.10%
Model saved at 
